In [1]:
import pandas as pd

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
old_df = pd.read_csv("../data/csv_oct09_24/sample_sep_1.csv")
old_df.columns

Index(['Unnamed: 0', 'responses_lo_1.2.institution_id',
       'responses_lo_1.2.class_id', 'responses_lo_1.2.student_id',
       'responses_lo_1.2.course_name', 'responses_lo_1.2.release',
       'responses_lo_1.2.book', 'responses_lo_1.2.response',
       'responses_lo_1.2.points_earned', 'responses_lo_1.2.completes_page',
       'responses_lo_1.2.dt_submitted', 'responses_lo_1.2.attempt',
       'responses_lo_1.2.id_p', 'responses_lo_1.2.chapter',
       'responses_lo_1.2.page', 'responses_lo_1.2.item_id',
       'responses_lo_1.2.item_type', 'responses_lo_1.2.lrn_type',
       'responses_lo_1.2.lrn_question_position',
       'responses_lo_1.2.lrn_question_reference', 'responses_lo_1.2.prompt',
       'responses_lo_1.2.lrn_option_0', 'responses_lo_1.2.lrn_option_1',
       'responses_lo_1.2.lrn_option_2', 'responses_lo_1.2.lrn_option_3',
       'responses_lo_1.2.lrn_option_4', 'responses_lo_1.2.lrn_option_5',
       'responses_lo_1.2.lrn_option_6', 'responses_lo_1.2.lrn_option_7',
 

In [2]:
new_df = pd.read_csv("../data/csv_oct15_24/responses_assessment_level2_3.1.csv")
new_df.columns

Index(['id_p', 'institution_id', 'class_id', 'course_name', 'release', 'book',
       'branch', 'student_id', 'item_id', 'item_type', 'chapter', 'page',
       'response', 'prompt', 'points_possible', 'points_earned',
       'dt_submitted', 'completes_page', 'attempt', 'user_agent',
       'lrn_session_id', 'lrn_response_id', 'lrn_activity_reference',
       'lrn_question_reference', 'lrn_question_position', 'lrn_type',
       'lrn_dt_started', 'lrn_dt_saved', 'lrn_status', 'lrn_response_json',
       'lrn_option_0', 'lrn_option_1', 'lrn_option_2', 'lrn_option_3',
       'lrn_option_4', 'lrn_option_5', 'lrn_option_6', 'lrn_option_7',
       'lrn_option_8', 'lrn_option_9', 'lrn_option_10', 'lrn_option_11',
       'chapter_num', 'page_num', 'level1', 'level2', 'level3', 'level4'],
      dtype='object')

In [3]:
len(new_df)

317633

In [7]:
import os
import rich

for p in os.listdir("../data/csv_oct15_24/"):
    df = pd.read_csv(f"../data/csv_oct15_24/{p}")
    rich.print(p, df["item_type"].unique())

responses_assessment_level2_3.2.csv ['learnosity']

responses_assessment_level2_3.3.csv ['code' 'learnosity']

responses_assessment_level2_1.1.csv ['learnosity']

responses_assessment_level2_1.3.csv ['learnosity']

responses_assessment_level2_3.1.csv ['learnosity' 'code']

...

responses_assessment_level2_2.2.csv ['learnosity' 'code']

responses_assessment_level2_2.3.csv ['code' 'learnosity']

responses_assessment_level2_4.1.csv ['learnosity' 'code']

responses_assessment_level2_4.2.csv ['learnosity' 'code']

responses_assessment_level2_4.3.csv ['learnosity' 'code']

In [5]:
new_df["item_type"].unique()

array(['learnosity', 'code'], dtype=object)

In [70]:
deduped_new_df = new_df.drop_duplicates()

In [8]:
import pandas as pd
import re
import rich
import os

from pyBKT.models import Model


def _extract_chapter(path: str) -> str:
    """
    Extract the chapter name from the path
    """
    # The path is in the format of responses_assessment_level2_1.1.csv
    # We want to extract the chapter number from the file name, i.e. 1.1 in this case
    chapter_pattern = re.compile(r".*responses_assessment_level2_(\d+\.\d+)\.csv")

    # Find column from data.columns that match the above pattern and extract out the chapter
    match = chapter_pattern.match(path)
    chapter = match.group(1)
    return chapter


def process_data(path: str, output_folder: str) -> dict[str, float]:
    """
    Process the data from the file specified by the path.
    Return the coefs learned from the model as a dict.
    """

    data = pd.read_csv(path, index_col=0, low_memory=False)
    chapter = _extract_chapter(path)

    rich.print(f"[yellow bold]Processing chapter: [/yellow bold]{chapter}")

    if len(data) == 0:
        rich.print("[red bold]No data found, skipped[/red bold]")
        return

    # Define the columns we're interested in
    points_earned_col = "points_earned"
    submitted_col = "dt_submitted"
    student_id_col = "student_id"
    item_id_col = "item_id"
    type_col = "item_type"

    # Run some transformations to get the data into the correct format
    # Drop nan values in the points_earned column
    data = data.dropna(subset=[points_earned_col])

    # Ensure the dt_submitted column is in datetime format
    data[submitted_col] = pd.to_datetime(data[submitted_col])

    # drop duplicate rows
    data = data.drop_duplicates()

    # Sort the data by student, item_id and submission time to create order_id
    data = data.sort_values(by=[student_id_col, item_id_col, submitted_col])

    # Process code submissions differently from learnosity
    def process_student_submissions(group):
        # Do not process if the question type is not `code`
        if group[type_col].iloc[0] != "code":
            return group

        # Always keep first submission
        result = group.iloc[[0]]

        # If first submission was incorrect and there exists a correct submission
        if (
            group[points_earned_col].iloc[0] == 0
            and (group[points_earned_col] > 0).any()
        ):
            # Find first correct submission
            first_correct = group[group[points_earned_col] > 0].iloc[0]
            result = pd.concat([result, first_correct.to_frame().T])

        return result

    # Apply the processing logic to each student-question combination
    processed_data = data.groupby(
        [student_id_col, item_id_col], group_keys=False
    ).apply(process_student_submissions)

    # Now sort by student and submission time to create final order_id
    processed_data = processed_data.sort_values(by=[student_id_col, submitted_col])

    # Create order_id (sequence number of observations per student)
    processed_data["order_id"] = processed_data.groupby(student_id_col).cumcount() + 1

    # Rename columns to match pyBKT expected format
    bkt_data = processed_data.rename(
        columns={
            student_id_col: "user_id",
            points_earned_col: "correct",
        }
    )

    # Assign a skill name (you can change this if you're tracking multiple skills)
    bkt_data["skill_name"] = (
        0  # If you're tracking only one skill, this can be constant
    )

    bkt_data = bkt_data.reset_index()

    # Drop unnecessary columns
    bkt_data = bkt_data[["user_id", "order_id", "correct", "skill_name"]]

    if len(bkt_data) == 0:
        rich.print("[red bold]No BKT data built, skipped[/red bold]")
        return

    # Fit the model
    model = Model(seed=42, num_fits=1)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)

        model.fit(data=bkt_data)

        # Generate predictions for each attempt
        predictions = model.predict(data=bkt_data)

    # Get the final predictions after all attempts for each student
    final_predictions = predictions.groupby("user_id").last()[
        ["correct_predictions", "state_predictions"]
    ]

    # Rename the aggregated columns
    final_predictions = final_predictions.rename(
        columns={
            "state_predictions": "final_state_predictions",
            "correct_predictions": "final_correct_predictions",
        }
    )

    # Create the folder if it doesn't exist using python
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    final_predictions.to_csv(f"{output_folder}/predictions_{chapter}.csv")

    # Process the coefficients
    coef = model.coef_["0"]
    return {
        "chapter": chapter,
        "prior": float(coef["prior"]),
        "learns": float(coef["learns"][0]),
        "guesses": float(coef["guesses"][0]),
        "slips": float(coef["slips"][0]),
    }

In [19]:
path = "../data/csv_oct15_24/responses_assessment_level2_2.2.csv"

data = pd.read_csv(path, index_col=0, low_memory=False)
chapter = _extract_chapter(path)

rich.print(f"[yellow bold]Processing chapter: [/yellow bold]{chapter}")

if len(data) == 0:
    rich.print("[red bold]No data found, skipped[/red bold]")

# Define the columns we're interested in
points_earned_col = "points_earned"
submitted_col = "dt_submitted"
student_id_col = "student_id"
item_id_col = "item_id"
type_col = "item_type"

# Run some transformations to get the data into the correct format
# Drop nan values in the points_earned column
data = data.dropna(subset=[points_earned_col])

# Ensure the dt_submitted column is in datetime format
data[submitted_col] = pd.to_datetime(data[submitted_col])

# drop duplicate rows
data = data.drop_duplicates()

# Sort the data by student, item_id and submission time to create order_id
data = data.sort_values(by=[student_id_col, item_id_col, submitted_col])


# Process code submissions differently from learnosity
def process_student_submissions(group):
    # Do not process if the question type is not `code`
    if group[type_col].iloc[0] != "code":
        return group

    # Always keep first submission
    result = group.iloc[[0]]

    # If first submission was incorrect and there exists a correct submission
    if group[points_earned_col].iloc[0] == 0 and (group[points_earned_col] > 0).any():
        # Find first correct submission
        first_correct = group[group[points_earned_col] > 0].iloc[0]
        result = pd.concat([result, first_correct.to_frame().T])

    return result


# Apply the processing logic to each student-question combination
processed_data = data.groupby([student_id_col, item_id_col], group_keys=False).apply(
    process_student_submissions
)

# Now sort by student and submission time to create final order_id
processed_data = processed_data.sort_values(by=[student_id_col, submitted_col])

# Create order_id (sequence number of observations per student)
processed_data["order_id"] = processed_data.groupby(student_id_col).cumcount() + 1

# Rename columns to match pyBKT expected format
bkt_data = processed_data.rename(
    columns={
        student_id_col: "user_id",
        points_earned_col: "correct",
    }
)

# Assign a skill name (you can change this if you're tracking multiple skills)
bkt_data["skill_name"] = 0  # If you're tracking only one skill, this can be constant

bkt_data = bkt_data.reset_index()

# Drop unnecessary columns
bkt_data = bkt_data[["user_id", "order_id", "correct", "skill_name"]]

Processing chapter: 2.2

In [20]:
data[data["item_type"] == "code"]

,institution_id,class_id,course_name,release,book,branch,student_id,item_id,item_type,chapter,...,lrn_option_8,lrn_option_9,lrn_option_10,lrn_option_11,chapter_num,page_num,level1,level2,level3,level4
id_p,,,,,,,,,,,,,,,,,,,,,
ch4-10 1,f390f0ca-5552-4d59-85e7-1bdede366fbe,e809a739-a6f8-458f-a6c9-960e81b74012,UCLATALL/czi-stats-course,v5.2,High School / Statistics and Data Science I (AB),release/v5.2,0025760d-ecfc-4578-9c7a-871a5a74391c,ch4-10,code,Chapter 4 - Explaining Variation,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2.20000,2.2.3,2.2.3.2
ch4-10 1,f390f0ca-5552-4d59-85e7-1bdede366fbe,e809a739-a6f8-458f-a6c9-960e81b74012,UCLATALL/czi-stats-course,v5.2,High School / Statistics and Data Science I (AB),release/v5.2,0025760d-ecfc-4578-9c7a-871a5a74391c,ch4-10,code,Chapter 4 - Explaining Variation,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2.20000,2.2.2,2.2.2.2
ch4-10 1,f390f0ca-5552-4d59-85e7-1bdede366fbe,e809a739-a6f8-458f-a6c9-960e81b74012,UCLATALL/czi-stats-course,v5.2,High School / Statistics and Data Science I (AB),release/v5.2,0025760d-ecfc-4578-9c7a-871a5a74391c,ch4-10,code,Chapter 4 - Explaining Variation,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2.20000,2.2.3,2.2.3.2
ch4-10 1,f390f0ca-5552-4d59-85e7-1bdede366fbe,e809a739-a6f8-458f-a6c9-960e81b74012,UCLATALL/czi-stats-course,v5.2,High School / Statistics and Data Science I (AB),release/v5.2,0025760d-ecfc-4578-9c7a-871a5a74391c,ch4-10,code,Chapter 4 - Explaining Variation,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2.20000,2.2.2,2.2.2.2
ch4-11 1,f390f0ca-5552-4d59-85e7-1bdede366fbe,e809a739-a6f8-458f-a6c9-960e81b74012,UCLATALL/czi-stats-course,v5.2,High School / Statistics and Data Science I (AB),release/v5.2,0025760d-ecfc-4578-9c7a-871a5a74391c,ch4-11,code,Chapter 4 - Explaining Variation,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2.20000,2.2.2,2.2.2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ch4-5 1,6496506d-b49b-4477-9435-ddea44a88008,926fc0d4-b7c4-4bb0-a6d7-3f0ca17c0ce2,UCLATALL/czi-stats-course,v5.5,High School / Statistics and Data Science I (AB),release/v5.5,ffe75b53-b7b6-4488-85f0-f60112e1dd43,ch4-5,code,Chapter 4 - Explaining Variation,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2.20000,2.2.3,2.2.3.3
ch4-5 1,6496506d-b49b-4477-9435-ddea44a88008,926fc0d4-b7c4-4bb0-a6d7-3f0ca17c0ce2,UCLATALL/czi-stats-course,v5.5,High School / Statistics and Data Science I (AB),release/v5.5,ffe75b53-b7b6-4488-85f0-f60112e1dd43,ch4-5,code,Chapter 4 - Explaining Variation,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2.20000,2.2.3,2.2.3.3
ch4-9 1,6496506d-b49b-4477-9435-ddea44a88008,926fc0d4-b7c4-4bb0-a6d7-3f0ca17c0ce2,UCLATALL/czi-stats-course,v5.5,High School / Statistics and Data Science I (AB),release/v5.5,ffe75b53-b7b6-4488-85f0-f60112e1dd43,ch4-9,code,Chapter 4 - Explaining Variation,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2.20000,2.2.3,2.2.3.2


In [24]:
# show at most 100 rows
pd.set_option("display.max_rows", 100)

In [28]:
data[data["student_id"] == "ffe75b53-b7b6-4488-85f0-f60112e1dd43"][
    ["item_id", "points_earned", "dt_submitted", "item_type"]
].drop_duplicates().sort_values(by="dt_submitted")

,item_id,points_earned,dt_submitted,item_type
id_p,,,,
ch2-20 1,ch2-20,0.00000,2024-03-18 16:40:38.000000,code
ch2-20 1,ch2-20,0.00000,2024-03-18 16:41:14.000000,code
ch2-20 1,ch2-20,0.00000,2024-03-18 16:41:45.000000,code
ch2-20 1,ch2-20,0.00000,2024-03-18 16:42:37.000000,code
Ch2_Structure_7 1,Ch2_Structure_7,1.00000,2024-03-18 16:44:14.379441,learnosity
A2_Review1_10 1,A2_Review1_10,1.00000,2024-03-21 16:44:33.961950,learnosity
A2_Review2_12 1,A2_Review2_12,0.00000,2024-03-21 17:05:55.607949,learnosity
A2_Review2_14 1,A2_Review2_14,1.00000,2024-03-21 17:13:37.446208,learnosity
Ch3_Concept_1 2,Ch3_Concept_1,0.00000,2024-03-22 16:50:21.868564,learnosity


In [34]:
processed_data[processed_data["student_id"] == "ffe75b53-b7b6-4488-85f0-f60112e1dd43"][
    ["item_id", "points_earned", "dt_submitted", "item_type", "order_id"]
]

,item_id,points_earned,dt_submitted,item_type,order_id
ch2-20 1,ch2-20,0.00000,2024-03-18 16:40:38,code,1
Ch2_Structure_7 1,Ch2_Structure_7,1.00000,2024-03-18 16:44:14.379441,learnosity,2
A2_Review1_10 1,A2_Review1_10,1.00000,2024-03-21 16:44:33.961950,learnosity,3
A2_Review2_12 1,A2_Review2_12,0.00000,2024-03-21 17:05:55.607949,learnosity,4
A2_Review2_14 1,A2_Review2_14,1.00000,2024-03-21 17:13:37.446208,learnosity,5
Ch3_Concept_1 2,Ch3_Concept_1,0.00000,2024-03-22 16:50:21.868564,learnosity,6
Ch3_Concept_5 1,Ch3_Concept_5,0.00000,2024-03-22 16:55:01.181651,learnosity,7
Ch3_Concept_10 1,Ch3_Concept_10,1.00000,2024-03-22 16:55:14.829937,learnosity,8
Ch3_Concept_6 1,Ch3_Concept_6,1.00000,2024-03-22 17:02:13.461422,learnosity,9
Ch3_Concept_2 1,Ch3_Concept_2,1.00000,2024-03-25 16:41:56.488562,learnosity,10


In [86]:
bkt_data[bkt_data["user_id"] == "0281beb5-c599-4da7-865f-af93088acfd9"]

,user_id,order_id,correct,skill_name,correct_predictions,state_predictions
244,0281beb5-c599-4da7-865f-af93088acfd9,1,0,0,0.57407,0.44480
245,0281beb5-c599-4da7-865f-af93088acfd9,2,0,0,0.48510,0.21820
246,0281beb5-c599-4da7-865f-af93088acfd9,3,1,0,0.43450,0.08933
247,0281beb5-c599-4da7-865f-af93088acfd9,4,0,0,0.46380,0.16395
248,0281beb5-c599-4da7-865f-af93088acfd9,5,1,0,0.42488,0.06482
...,...,...,...,...,...,...
283,0281beb5-c599-4da7-865f-af93088acfd9,40,0,0,0.45599,0.14407
284,0281beb5-c599-4da7-865f-af93088acfd9,41,0,0,0.42154,0.05632
285,0281beb5-c599-4da7-865f-af93088acfd9,42,0,0,0.40789,0.02154
286,0281beb5-c599-4da7-865f-af93088acfd9,43,0,0,0.40291,0.00888


In [81]:
data.columns

Index(['institution_id', 'class_id', 'course_name', 'release', 'book',
       'branch', 'student_id', 'item_id', 'item_type', 'chapter', 'page',
       'response', 'prompt', 'points_possible', 'points_earned',
       'dt_submitted', 'completes_page', 'attempt', 'user_agent',
       'lrn_session_id', 'lrn_response_id', 'lrn_activity_reference',
       'lrn_question_reference', 'lrn_question_position', 'lrn_type',
       'lrn_dt_started', 'lrn_dt_saved', 'lrn_status', 'lrn_response_json',
       'lrn_option_0', 'lrn_option_1', 'lrn_option_2', 'lrn_option_3',
       'lrn_option_4', 'lrn_option_5', 'lrn_option_6', 'lrn_option_7',
       'lrn_option_8', 'lrn_option_9', 'lrn_option_10', 'lrn_option_11',
       'chapter_num', 'page_num', 'level1', 'level2', 'level3', 'level4',
       'order_id'],
      dtype='object')

In [87]:
student_data = data[data["student_id"] == "0281beb5-c599-4da7-865f-af93088acfd9"][
    ["student_id", "points_earned", "dt_submitted"]
]
student_data

,student_id,points_earned,dt_submitted
id_p,,,
B1_Review1_09 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2023-12-04 17:54:37.109867
B1_Review1_11 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2023-12-04 17:55:11.702449
B1_Review1_12 1,0281beb5-c599-4da7-865f-af93088acfd9,1.00000,2023-12-04 17:55:23.662697
B1_Review1_13 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2023-12-04 17:55:42.314275
B1_Review2_05 1,0281beb5-c599-4da7-865f-af93088acfd9,1.00000,2023-12-04 18:11:01.373617
...,...,...,...
B4_Review1_16 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2024-03-19 16:34:34.053801
B4_Review1_19 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2024-03-19 16:34:52.080108
B4_Review2_06 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2024-03-19 16:36:11.040208


In [88]:
student_data.drop_duplicates()

,student_id,points_earned,dt_submitted
id_p,,,
B1_Review1_09 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2023-12-04 17:54:37.109867
B1_Review1_11 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2023-12-04 17:55:11.702449
B1_Review1_12 1,0281beb5-c599-4da7-865f-af93088acfd9,1.00000,2023-12-04 17:55:23.662697
B1_Review1_13 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2023-12-04 17:55:42.314275
B1_Review2_05 1,0281beb5-c599-4da7-865f-af93088acfd9,1.00000,2023-12-04 18:11:01.373617
...,...,...,...
B4_Review1_16 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2024-03-19 16:34:34.053801
B4_Review1_19 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2024-03-19 16:34:52.080108
B4_Review2_06 1,0281beb5-c599-4da7-865f-af93088acfd9,0.00000,2024-03-19 16:36:11.040208


In [82]:
model = Model(seed=42, num_fits=1)

In [83]:
import warnings

# Silence RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

model.fit(data=bkt_data)

In [84]:
prediction_df = model.predict(data=bkt_data)

In [89]:
pd.set_option("display.max_rows", 50)

prediction_df[prediction_df["user_id"] == "0281beb5-c599-4da7-865f-af93088acfd9"]

,user_id,order_id,correct,skill_name,correct_predictions,state_predictions
244,0281beb5-c599-4da7-865f-af93088acfd9,1,0,0,0.57407,0.44480
245,0281beb5-c599-4da7-865f-af93088acfd9,2,0,0,0.48510,0.21820
246,0281beb5-c599-4da7-865f-af93088acfd9,3,1,0,0.43450,0.08933
247,0281beb5-c599-4da7-865f-af93088acfd9,4,0,0,0.46380,0.16395
248,0281beb5-c599-4da7-865f-af93088acfd9,5,1,0,0.42488,0.06482
249,0281beb5-c599-4da7-865f-af93088acfd9,6,1,0,0.44733,0.12200
250,0281beb5-c599-4da7-865f-af93088acfd9,7,0,0,0.48465,0.21706
251,0281beb5-c599-4da7-865f-af93088acfd9,8,0,0,0.43429,0.08879
252,0281beb5-c599-4da7-865f-af93088acfd9,9,1,0,0.41274,0.03392
253,0281beb5-c599-4da7-865f-af93088acfd9,10,1,0,0.42547,0.06633


In [11]:
model.coef_

{'0': {'prior': np.float64(0.2596580956384623),
  'learns': array([0.00368407]),
  'guesses': array([0.36249751]),
  'slips': array([0.17124273]),
  'forgets': array([0.])}}

In [17]:
coefs = []

coef = model.coef_["0"]
coef

{'prior': np.float64(0.259701541449756),
 'learns': array([0.00368414]),
 'guesses': array([0.36248229]),
 'slips': array([0.17126229]),
 'forgets': array([0.])}

In [18]:
float(coef["prior"])

0.259701541449756

In [20]:
coef["learns"][0]

np.float64(0.003684136079713055)

In [21]:
float(coef["learns"][0])

0.003684136079713055

In [23]:
{
    "chapter": chapter,
    "prior": float(coef["prior"]),
    "learns": float(coef["learns"][0]),
    "guesses": float(coef["guesses"][0]),
    "slips": float(coef["slips"][0]),
}

{'chapter': '3.2',
 'prior': 0.259701541449756,
 'learns': 0.003684136079713055,
 'guesses': 0.3624822871232267,
 'slips': 0.17126229068293614}

In [26]:
coef_df = pd.DataFrame(
    [
        {
            "chapter": chapter,
            "prior": float(coef["prior"]),
            "learns": float(coef["learns"][0]),
            "guesses": float(coef["guesses"][0]),
            "slips": float(coef["slips"][0]),
        },
        {
            "chapter": "1.1",
            "prior": float(coef["prior"]),
            "learns": float(coef["learns"][0]),
            "guesses": float(coef["guesses"][0]),
            "slips": float(coef["slips"][0]),
        },
        {
            "chapter": "1.2",
            "prior": float(coef["prior"]),
            "learns": float(coef["learns"][0]),
            "guesses": float(coef["guesses"][0]),
            "slips": float(coef["slips"][0]),
        },
    ]
)

In [27]:
coef_df.sort_values(by="chapter")

,chapter,prior,learns,guesses,slips
1,1.1,0.25970,0.00368,0.36248,0.17126
2,1.2,0.25970,0.00368,0.36248,0.17126
0,3.2,0.25970,0.00368,0.36248,0.17126
